https://towardsdatascience.com/multi-task-learning-with-pytorch-and-fastai-6d10dc7ce855


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#%cd /content/gdrive/My\ Drive/Colab/IDL_Final_Project
%cd /content/gdrive/My\ Drive/project/IDL_Final_Project

[Errno 2] No such file or directory: '/content/gdrive/My Drive/project/IDL_Final_Project'
/content


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import librosa
import librosa.display
import IPython.display as ipd

import numpy as np
import matplotlib.pyplot as plt

import os
from datetime import datetime
import time

### Unzipping Fisher

In [ ]:
!pwd

/content


In [ ]:
# !cp -r '/content/gdrive/My Drive/Colab/IDL_Final_Project/fisher/vad_wavs.zip' '/content/gdrive/My Drive/Colab/IDL_Final_Project/fisher_wav'

In [ ]:
# from zipfile import ZipFile

# # with ZipFile('fisher/vad_wavs.zip', 'r') as zip:
# #   # zip.printdir()
# #   zip.extractall()
# #   print('done!')

# with open('fisher/vad_wavs.zip', 'rb') as zf:
#     z = zipfile.ZipFile(zf)
#     for member in z.infolist():
#         try:
#             z.extract(member)
#         except zipfile.error as e:
#           print(e)
#           # print("Error".format(e, member.filename))
#             # log the error, the member.filename, whatever

In [ ]:
# !7za x "fisher/vad_wavs.zip" #-d "drive/My Drive/PATH_TO_OUTPUT"

### Audio

In [ ]:
ipd.Audio('train/dr1/fcjf0/sx217.wav') #train/dr6/fbch0/sa2.wav

ValueError: ignored

In [ ]:
ipd.Audio('test/dr1/faks0/sa1.wav') #train/dr6/fbch0/sa2.wav

In [ ]:
ipd.Audio('./vad_wavs/FE_03_00287.ch1.wav')

## Generating Labels

### Training

Multi-Task Deep Recurrent Neural Networks: http://59.108.48.5/struct/Projects/multitask_OAD.html

### Extract MFCCs

In [ ]:
def extract_mfcc(audio_file):
  
  n_mels = 64
  fmin = 100
  fmax = 8000

  # Size of the FFT, which will also be used as the window length
  n_fft = 100 #1024 100

  # Step or stride between windows. If the step is smaller than the window length, the windows will overlap
  hop_length = 90#512

  y, sr = librosa.load(audio_file)

  
  mel = librosa.feature.melspectrogram(y, sr=sr, n_fft=n_fft, hop_length=hop_length,
                                       n_mels=n_mels, fmin = fmin,fmax=fmax)

  return mel

In [ ]:
extract_mfcc('train/dr1/fcjf0/sx217.wav').shape

### Converting date

In [ ]:
# def convert_date(date):
#   date = date.split('/')
#   year = 1900 + int(date[2])
#   month = int(date[0])
#   day = int(date[1])

#   return datetime(year, month, day)

### Extract Speaker Info

In [ ]:
# Extracting Speaker Info

speaker_info = {}

with open('doc/speaker_info.txt', 'r') as reader:
  # print("\tData: ID  Sex DR Use  RecDate   BirthDate  Ht    Race Edu  Comments")
  for idx,line in enumerate(reader):
    keys = line.split()
    speaker_id = keys[0]
    gender = keys[1]
    dr = keys[2]

    rec_date = keys[4]
    birth_date = keys[5]
    height = keys[6]

    height = height[:-1].split('\'')

    feet = int(height[0])

    inches = 0
    if len(height) == 2:
      inches = int(height[1])

    ht_in_cm = feet * 30.48 + inches * 2.54  # Converting from Feet and inches to centimeters

    # break
    # print(birth_date.split('/')[-1], rec_date.split('/')[-1])

    try:
      birth_year = int(birth_date.split('/')[-1]) #convert_date(birth_date)
      rec_year = int(rec_date.split('/')[-1]) #convert_date(rec_date)
    except ValueError:
      continue

    # time_delta = (rec_date - birth_date)
    # age = time_delta.days # / 365
    age = rec_year - birth_year

    speaker_info[speaker_id] = {'DR': dr, 'Gender': gender, 
                                'Age': age,'Ht': ht_in_cm }

In [ ]:
speaker_info['daw0'.upper()] #fdaw0

### Get all .wav filenames in a directory

In [ ]:
def get_filenames(dir):
    matches = []
    for root, dirnames, filenames in os.walk(dir):
        for filename in filenames:
            if filename.endswith(('.wav')):
                matches.append(os.path.join(root, filename))
    return matches

### Extracting speaker data

In [ ]:
def extract_speaker_data(dir):
  # dir = 'train/dr6/fbch0'
  audio_filnames = get_filenames(dir)

  dev_data = []
  for audio_filename in audio_filnames:
    mel = extract_mfcc(audio_filename)
    dev_data.append(mel)

  return dev_data

### Extract Train data

In [ ]:
def extract_data_from_directory(directory):
  # Extracting training data
  keys = []
  data = []
  labels = []

  for root, dirs, files in os.walk(directory):
    path_keys = root.split('/')

    if len(path_keys) == 3:
      dr = path_keys[1][-1]
      id = path_keys[2]
      gender = id[0]
      speaker_id = id[1:].upper()

      if speaker_id not in speaker_info.keys():
        print('Speaker id \'{}\' not in speaker_info'.format(speaker_id))
        continue

      keys.append({'DR': dr, 'Gender: ': gender, 'Speaker_Id: ': speaker_id})


      # mfcc_data = extract_speaker_data(root)
      # data.extend(mfcc_data)

      try:
        mfcc_data = extract_speaker_data(root)
        data.extend(mfcc_data)
      except EOFError as e:
        print(e)
        print('Error occurred with file: ',root)
        continue

      age = speaker_info[speaker_id]['Age']
      height = speaker_info[speaker_id]['Ht']
      gender = 0 if speaker_info[speaker_id]['Gender'] == 'F' else 1

      labels.extend([[age, height, gender]] * len(mfcc_data))

    else:
        print(root)

  return  data, labels

In [ ]:
t0 = time.time()
train_data, train_labels = extract_data_from_directory('train')
print("Time:",(time.time() - t0))

In [ ]:
train_data[0:2]

In [ ]:
print(len(train_data))
print(len(train_labels))#.shape)

In [ ]:
train_data[0].shape

In [ ]:
train_data[1].shape

In [ ]:
t0 = time.time()
test_data, test_labels = extract_data_from_directory('test')
print("Time:",(time.time() - t0))

In [ ]:
# print(test_data.shape)
# print(test_labels.shape)

### Saving numpy array

In [ ]:
train = [np.transpose(mel) for mel in train_data]
test = [np.transpose(mel) for mel in test_data]
train = np.array(train)
test = np.array(test)

In [ ]:
# Training data
'''np.save('train_data', train)
np.save('train_labels', train_labels)

# Test data
np.save('test_data', test)
np.save('test_labels', test_labels)'''
np.save('train_data1', train)
np.save('train_labels1', train_labels)

# Test data
np.save('test_data1', test)
np.save('test_labels1', test_labels)

In [ ]:
speaker_info['DAW0']

In [ ]:
test_labels[0]